In [81]:
from numpy import array,identity,diagonal
import numpy as np
from math import sqrt
import math

def jacobi(a,tol = 1.0e-9): # Jacobi method
 
    def maxElem(a): # Find largest off-diag. element a[k,l]
        n = len(a)
        aMax = 0.0
        for i in range(n-1):
            for j in range(i+1,n):
                if abs(a[i,j]) >= aMax:
                    aMax = abs(a[i,j])
                    k = i; l = j
        return aMax,k,l
 
    def rotate(a,p,k,l): # Rotate to make a[k,l] = 0
        
        n = len(a)
        aDiff = a[l,l] - a[k,k]
        if abs(a[k,l]) < abs(aDiff)*1.0e-36: t = a[k,l]/aDiff
        else:
            phi = aDiff/(2.0*a[k,l])
            t = 1.0/(abs(phi) + sqrt(phi**2 + 1.0))
            if phi < 0.0: t = -t
        c = 1.0/sqrt(t**2 + 1.0); s = t*c
        tau = s/(1.0 + c)
        temp = a[k,l]
        a[k,l] = 0.0
        a[k,k] = a[k,k] - t*temp
        a[l,l] = a[l,l] + t*temp
        for i in range(k):      # Case of i < k
            temp = a[i,k]
            a[i,k] = temp - s*(a[i,l] + tau*temp)
            a[i,l] = a[i,l] + s*(temp - tau*a[i,l])
        for i in range(k+1,l):  # Case of k < i < l
            temp = a[k,i]
            a[k,i] = temp - s*(a[i,l] + tau*a[k,i])
            a[i,l] = a[i,l] + s*(temp - tau*a[i,l])
        for i in range(l+1,n):  # Case of i > l
            temp = a[k,i]
            a[k,i] = temp - s*(a[l,i] + tau*temp)
            a[l,i] = a[l,i] + s*(temp - tau*a[l,i])
        for i in range(n):      # Update transformation matrix
            temp = p[i,k]
            p[i,k] = temp - s*(p[i,l] + tau*p[i,k])
            p[i,l] = p[i,l] + s*(temp - tau*p[i,l])
        print("=" * 10)
        print(p)
        print("=" * 10)
 
    n = len(a)
    maxRot = 5*(n**2)       # Set limit on number of rotations
    p = identity(n)*1.0     # Initialize transformation matrix
    
    for i in range(maxRot): # Jacobi rotation loop 
        aMax,k,l = maxElem(a)
        if aMax < tol: return diagonal(a),p
        rotate(a,p,k,l)
    print('Jacobi method did not converge')

In [94]:
a = np.array([[6.81, 1.04, 1.03, 1.165], 
              [1.04, 3.61, 1.3, 0.16],
              [1.03, 1.3, 5.99, 2.1],
              [1.165, 0.16, 2.1, 5.55]])

In [87]:
jacobi(a)

[[ 1.          0.          0.          0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          0.74303153 -0.66925641]
 [ 0.          0.          0.66925641  0.74303153]]
[[ 0.81474593  0.          0.57981814  0.        ]
 [ 0.          1.          0.          0.        ]
 [-0.43082316  0.          0.60538191 -0.66925641]
 [-0.38804701  0.          0.54527393  0.74303153]]
[[ 0.81474593 -0.14426587  0.56158386  0.        ]
 [ 0.          0.96855173  0.24881228  0.        ]
 [-0.43082316 -0.15062645  0.5863437  -0.66925641]
 [-0.38804701 -0.13567085  0.52812601  0.74303153]]
[[ 0.81474593 -0.11511408  0.56158386  0.08695626]
 [ 0.          0.77283655  0.24881228 -0.58379458]
 [-0.43082316 -0.5235837   0.5863437  -0.44322972]
 [-0.38804701  0.33960642  0.52812601  0.67466281]]
[[ 0.80209396 -0.18359589  0.56158386  0.08695626]
 [ 0.06535028  0.77006862  0.24881228 -0.58379458]
 [-0.47355387 -0.4852785   0.5863437  -0.44322972]
 [-0.35794043  0.37120298  

(array([ 5.73204313,  2.63911924,  9.36267962,  4.22615802]),
 array([[ 0.79295028, -0.17894702,  0.57322719,  0.10304565],
        [ 0.06809224,  0.77246307,  0.25109253, -0.57932443],
        [-0.4773721 , -0.479946  ,  0.58970674, -0.44046984],
        [-0.37243681,  0.37540726,  0.51049887,  0.67804949]]))

In [146]:
def max_not_diagonal(matrix):
    a = np.array(matrix)
    i,j = np.indices(a.shape)
    a[i >= j] = 0
    index = np.unravel_index(np.abs(a).argmax(), a.shape)
    return index, a[index]

def build_rotate_matrix(a_k, index):
    i, j = index
    U = np.zeros(a_k.shape)
    if abs(a_k[i, i] - a_k[j, j]) < 0.0000001:
        print("FUCK" * 10)
        phi = math.phi / 4.0
    else:
        phi = 0.5 * math.atan(2.0 * a_k[i, j] / (a_k[i, i] - a_k[j, j]))
    U[i, j] = - math.sin(phi)
    U[j, i] = math.sin(phi)
    U[i, i] = math.cos(phi)
    U[j, j] = math.cos(phi)
    for m in range(len(U)):
        if m != i and m != j:
            U[m, m] = 1
    return U
    
def calc_stop(matrix):
    i,j = np.indices(matrix.shape)
    return (matrix[i < j] ** 2).sum() ** 0.5

def print_not_diagonal(matrix):
    i,j = np.indices(matrix.shape)
    print(matrix[i < j])

In [149]:
input_matrix = np.array([[6.81, 1.04, 1.03, 1.165], 
                   [1.04, 3.61, 1.3, 0.16],
                   [1.03, 1.3, 5.99, 2.1],
                   [1.165, 0.16, 2.1, 5.55]])

max_iteration = 50

for number_iteration in range(max_iteration):
    if number_iteration == 0:    
        Ak = np.array(input_matrix)
    index, element = max_not_diagonal(Ak)
    U = build_rotate_matrix(Ak, index)        
    Ak = np.dot(U.T, Ak).dot(U)
    if calc_stop(Ak) < 0.0001:
        break
else:
    print("FUCK" * 100)
    

In [125]:
index1, ele1 = max_not_diagonal(A1)
print(index1, ele1)
print(A1)
print(A1[index1])
U1 = build_rotate_matrix(A1, index1)
print(U1)

A2 = np.dot(U1.T, A1).dot(U1)
print(A2)
print(A2[index1])

(0, 2) 1.54500619617
[[  6.81000000e+00   1.04000000e+00   1.54500620e+00   1.76297628e-01]
 [  1.04000000e+00   3.61000000e+00   1.07302202e+00  -7.51148291e-01]
 [  1.54500620e+00   1.07302202e+00   7.88149236e+00   0.00000000e+00]
 [  1.76297628e-01  -7.51148291e-01   2.22044605e-16   3.65850764e+00]]
1.54500619617
[[ 0.81474593  0.          0.57981814  0.        ]
 [ 0.          1.          0.          0.        ]
 [-0.57981814  0.          0.81474593  0.        ]
 [ 0.          0.          0.          1.        ]]
[[  5.71048837e+00   2.25178131e-01  -8.88178420e-16   1.43637774e-01]
 [  2.25178131e-01   3.61000000e+00   1.47725118e+00  -7.51148291e-01]
 [  0.00000000e+00   1.47725118e+00   8.98100400e+00   1.02220563e-01]
 [  1.43637774e-01  -7.51148291e-01   1.02220563e-01   3.65850764e+00]]
-8.881784197e-16
